Leung Wai Liu <br>
JPMC-SMM4H <br>
July 14, 2022 <br>
Task 2a SEPARATE Weighted Ensembling

In [4]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
from training_code import calculate_overall_performance_metrics
import os

In [5]:
# Loading up all the predictions data

n_rnds = 5
original_df = pd.read_csv('../Datasets/dev.tsv', sep='\t')
models = ['bert-large-uncased', 'roberta-large']
n_models = len(models)

epoch_string = '../15_epochs_large_model/eval_testing/saved_eval_test_result_2b'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

# Loading up all of the results
best_f1 = pd.read_csv('../15_epochs_large_model/eval_testing/validation_stats/all_best_overall_f1_score.tsv', sep='\t')
best_f1



,Unnamed: 0,bert-large-uncased,roberta-large
0,0,0.788181,0.805198
1,1,0.798593,0.789957
2,2,0.771915,0.777964
3,3,0.813640,0.796570
4,4,0.784882,0.782647


In [6]:
# Retrieving all the predictions from the 
list_of_df = pd.DataFrame(index=range(n_rnds), columns=models)
list_of_f1_score = []
sum_of_all_f1_score_list = pd.DataFrame(index=[0], columns=models)

for model in models: 
    specific_model_row = []
    specific_model_row_f1 = []
    sum_of_all_f1_score = 0
    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_eval_test_result.tsv'
        
        specific_f1_score = best_f1.at[rnd, model]
        specific_model_row_f1.append(specific_f1_score)
        sum_of_all_f1_score += specific_f1_score
        
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        list_of_df.at[rnd, model] = particular_model_df
    sum_of_all_f1_score_list.at[0,model] = sum_of_all_f1_score

sum_of_all_f1_score_list

,bert-large-uncased,roberta-large
0,3.95721,3.95234


In [18]:
# TAKING THE MAJORITY OF DATA

weighted_avg_original_tweet_id_list = []
weighted_avg_original_sentence_list = []
weighted_avg_original_claim_list = []
weighted_avg_original_label_list = []

weighted_avg_predicted_number_results = pd.DataFrame(index=range(n_rows), columns=models)
weighted_avg_predicted_results = pd.DataFrame(index=range(n_rows), columns=models)


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['id']
    original_sentence = row['Tweet']
    original_claim = row['Claim']
    original_label = row['Premise']
    
    # transferring the labels over to final list
    weighted_avg_original_tweet_id_list.append(original_tweet_id)
    weighted_avg_original_sentence_list.append(original_sentence)
    weighted_avg_original_claim_list.append(original_claim)
    weighted_avg_original_label_list.append(original_label)
    
    # go through every models' row of data 
    
    for model in models:
        specific_row_value = 0.0
        for rnd_num in range(n_rnds):
            # print(particular_df)
            particular_df = list_of_df.at[rnd_num, model]
            row = particular_df.loc[(particular_df['tweet_id'] == original_tweet_id) & (particular_df['text'] == original_sentence)]
            
            # prediction = row['Premise'].values[0] * list_of_f1_score[model_num][rnd_num]
            specific_row_value += prediction
        
        
        specific_row_value = specific_row_value / (sum_of_all_f1_score_list.at[0, model])
       
        specific_row_result = int(specific_row_value)
        
        # weighted_avg_predicted_results.at[index, model] = specific_row_label
        weighted_avg_predicted_number_results.at[index, model] = specific_row_result
    

In [19]:
weighted_avg_predicted_number_results

,dccuchile/bert-base-spanish-wwm-uncased,dccuchile/bert-base-spanish-wwm-cased,xlm-roberta-base,bert-base-multilingual-uncased,bert-base-multilingual-cased
0,2,2,2,2,2
1,2,2,2,2,2
2,2,2,2,2,2
3,2,2,2,2,2
4,2,2,2,2,2
...,...,...,...,...,...
5020,2,2,2,2,2
5021,2,2,2,2,2
5022,0,0,0,0,0
5023,2,2,2,2,2


In [21]:
# Calculating sklearn metrics

weighted_avg_original_numbered_label_list = [labels_to_ids[label] for label in weighted_avg_original_label_list]
#weighted_avg_predicted_number_results = [labels_to_ids[label] for label in weighted_avg_predicted_results]

sep_weighted_avg_metrics = pd.DataFrame(index=['f1', 'accuracy', 'precision', 'recall'], columns=models)

for model in models: 
        sep_maj_predicted_number_results = weighted_avg_predicted_number_results[model].tolist() 
        accuracy_result = accuracy_score(weighted_avg_original_numbered_label_list, sep_maj_predicted_number_results)
        f1_result = f1_score(weighted_avg_original_numbered_label_list, sep_maj_predicted_number_results, labels=[0], average=None)[0]
        precision_result = precision_score(weighted_avg_original_numbered_label_list, sep_maj_predicted_number_results, labels=[0], average=None)[0]
        recall_result = recall_score(weighted_avg_original_numbered_label_list, sep_maj_predicted_number_results, labels=[0], average=None)[0]

        sep_weighted_avg_metrics.at['f1', model] = f1_result
        sep_weighted_avg_metrics.at['accuracy', model] = accuracy_result
        sep_weighted_avg_metrics.at['precision', model] = precision_result
        sep_weighted_avg_metrics.at['recall', model] = recall_result

print(sep_weighted_avg_metrics)

# Saving results to file
os.makedirs('../15_epochs_small_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble', exist_ok=True)

sep_weighted_avg_metrics.to_csv('../15_epochs_small_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble/sep_weighted_avg_metrics.tsv', sep='\t')





          dccuchile/bert-base-spanish-wwm-uncased  \
f1                                       0.723274   
accuracy                                 0.827861   
precision                                0.623457   
recall                                   0.861145   

          dccuchile/bert-base-spanish-wwm-cased xlm-roberta-base  \
f1                                     0.732424          0.73947   
accuracy                               0.831841         0.834428   
precision                              0.643318         0.645191   
recall                                 0.850183         0.866017   

          bert-base-multilingual-uncased bert-base-multilingual-cased  
f1                              0.740741                      0.74047  
accuracy                        0.836219                     0.837214  
precision                        0.64114                      0.64808  
recall                          0.876979                     0.863581  


In [ ]:
# Saving it as a dataframe

for model in models:
    weighted_avg_predicted_results = [ids_to_labels[id] for id in weighted_avg_predicted_number_results[model].tolist()]
    unformatted_weighted_avg_prediction_data = pd.DataFrame(zip(weighted_avg_original_tweet_id_list, weighted_avg_original_sentence_list, weighted_avg_original_label_list, weighted_avg_predicted_results), columns=['tweet_id', 'text', 'Orig', 'label'])
    formatted_weighted_avg_prediction_data = unformatted_weighted_avg_prediction_data.drop(columns = ['text', 'Orig'])

    to_save_location = '../15_epochs_small_model/eval_testing/sep_weighted_avg_results/' + model + '/'
    os.makedirs(to_save_location, exist_ok=True)
    unformatted_to_save_location = to_save_location + 'unformatted_sep_majority.tsv'
    formatted_to_save_location = to_save_location + 'formatted_sep_majority.tsv'
    
    unformatted_weighted_avg_prediction_data.to_csv(unformatted_to_save_location, sep='\t', index=False)
    formatted_weighted_avg_prediction_data.to_csv(formatted_to_save_location, sep='\t', index=False)
